In [0]:
# dbfs:/FileStore/shared_uploads/kumar@vcloudmatesolutions.com/products.csv
# dbfs:/FileStore/shared_uploads/kumar@vcloudmatesolutions.com/sales_orders.csv

In [0]:
sales_ordersDF = spark.read.csv('dbfs:/FileStore/shared_uploads/kumar@vcloudmatesolutions.com/sales_orders.csv',header=True,inferSchema=True)
productsDF     = spark.read.csv('dbfs:/FileStore/shared_uploads/kumar@vcloudmatesolutions.com/products.csv',header=True,inferSchema=True)

In [0]:
display(sales_ordersDF)

In [0]:
display(productsDF)

In [0]:
display(sales_ordersDF.isStreaming)
display(productsDF.isStreaming)

In [0]:
sales_ordersDF.write.format('parquet').save('/tmp/salesorders')
productsDF.write.format('parquet').save('/tmp/products')

In [0]:
sales_orders_StreamingDF = spark.readStream.format('parquet') \
                                            .schema(sales_ordersDF.schema) \
                                            .load('/tmp/salesorders')

In [0]:
products_StreamingDF = spark.readStream.format('parquet') \
                                            .schema(productsDF.schema) \
                                            .load('/tmp/products')

In [0]:
display(sales_orders_StreamingDF.isStreaming)
display(products_StreamingDF.isStreaming)

In [0]:
join_sales_orders_products_StreamingDF = sales_orders_StreamingDF.join(products_StreamingDF,'product_id')

In [0]:
#q = df.writeStream.format("console").start()
#time.sleep(3)
#q.stop()

In [0]:
# query = join_sales_orders_products_StreamingDF.writeStream.format('console').start()

In [0]:
source_checkpointLocation = '/tmp/_checkpoint'
tableName = 'StreamingSalesOrderProductsTable'

In [0]:
query = join_sales_orders_products_StreamingDF.writeStream.format('parquet') \
                                                .option("checkpointLocation",source_checkpointLocation) \
                                                .outputMode('append') \
                                                .table(tableName)
                                                
                                                

In [0]:
totalSalesQuery ='SELECT SUM(total_amount) FROM streamingsalesorderproductstable'

In [0]:
totalSalesQueryResults = spark.sql(totalSalesQuery)

In [0]:
display(totalSalesQueryResults)

In [0]:
# query.awaitTermination()